# pygsf 3 - geology

*March-April, 2018, Mauro Alberti, alberti.m65@gmail.com*

Developement code:

In [1]:
%load_ext autoreload
%autoreload 1

## 1. Fault data vs. focal mechanisms

Structural investigations of brittle regimes rock volumes deal with fault data derived from field measurements of meso-fault surfaces. Seismic events, generated by active faults, are generally represented as focal mechanisms.

Due to their common physical origin, meso-fault data and focal mechanisms share common aspects, even if it is useful to treat them as two distinct type structures, also from an algorithmic point of view.

In *gsfpy* fault data types are stored in the *faults* submodule:

In [2]:
from pygsf.geology.faults import *

We use also matplotlib inline and mplstereonet for plotting stereonet: 

In [3]:
%matplotlib inline

## 2. Fault data

Fault geometry characteristics can be defined using the concept of a geological plane, approximating the local fault surface, and of a slickenline, basically a vector that is parallel to the movement orientation. A slickenline parameters are its orientation and, when recognised, a movement direction.  

In the algorithms, slickenlines are represented by the **Slick** class, that stores the slickenline orientation and movement direction. A meso-fault datum is represented by the **Fault** class, that is composed of a **Plane** instance (i.e., the geological plane), and a **Slick** instance.

### 2.1 Slickenlines

A slickenline is created via a **Slick** instance. The input parameters are: *trend*, *plunge* and whether movement direction is known or not.

In [4]:
slick_k = Slick(140, 15)  # slickenline with known, downward movement direction

In [5]:
slick_k

Slick(az: 140.00°, pl: 15.00°, known_dir: True, time: 0.0)

In the above example, the slickenline movement direction is towards N140°, with a positive dip angle value of 15°, i.e., pointing downward. In the print output, *True* indicates that the movement sense is known.

In [6]:
slick_k.hasKnownSense

True

We can create a slickenline object with unknown movement sense by explicitely setting the *known* flag to **False** (default value is **True**), as in the following example:

In [7]:
slick_u = Slick(210, 60, known=False)  # slickenkline with unknown movement sense

In [8]:
slick_u

Slick(az: 210.00°, pl: 60.00°, known_dir: False, time: 0.0)

In [9]:
slick_u.hasKnownSense

False

It is possible to convert an unknown movement to a known movement or viceversa, and, when the movement is known, to invert it, with the *setKnownSense()* and *setUnknownSense()* methods.

### 2.2 Fault planes with slickenlines

Having described slickenlines, we can now consider how to represent fault planes presenting slickenlines.

An example is:

In [10]:
flt = Fault(90, 45, slickenlines=[Slick(90, 45, known=False)])

A fault instance is created by using the **Fault** class, initialized with two parameters:
- the geological plane: a **Plane** instance
- the slickenlines: zero, one or more **Slick** instances
Note that there is no required 1 : 1 relationship between a fault plane and a slickenline. A fault plane can contain no slickeline at all, or even more than a single slickenline, for instance because the movements on it were curvilinear or because the history of movements is polyphasic. 

In [11]:
type(flt)  # we check the type

pygsf.geology.faults.Fault

We check if this fault instance has a known movement sense:

In [12]:
flt.knownSense

False

Since the slickenline orientation was defined via a **Axis** instance, the movement sense is not known.

## 3. Focal mechanisms and P-T-B axes 

Classes for P-T-B axes are defined in the *ptbaxes* submodule:

In [13]:
from pygsf.geology.ptbaxes import *

P-T axes instances can be created using the **PTBAxes** class.

We can initialize an instance in four ways: 
- by providing a *couple of **Axis***, representing the T and P axes
- by giving a **Fault** instance as input parameter
- defining a couple of Cartesian vectors, representing the P and T axes
- by providing a quaternion.

Creation of a P-T axes instance from a couple of Axis:

In [14]:
ptbx_from_axes = PTBAxes(p_axis=Axis.fromAzPl(0, 0), t_axis=Axis.fromAzPl(90, 0))

In [15]:
print(ptbx_from_axes)

PTBAxes(P Axis(az: 360.00°, pl: -0.00°), T Axis(az: 90.00°, pl: 0.00°))


Creation of a P-T axes instance from a FaultSlick instance:

In [16]:
ptbx_from_fltsl = PTBAxes.fromFaultSlick(Fault(90, 45, slickenlines=[Slick(90, 45)]))

In [17]:
print(ptbx_from_fltsl)

PTBAxes(P Axis(az: 360.00°, pl: -90.00°), T Axis(az: 90.00°, pl: -0.00°))


An example of creating a P-T axes instance from a couple of Vect instances is:

In [18]:
ptbx_from_vects = PTBAxes.fromVects(t_vector=Vect(1,0,0), p_vector=Vect(0,1,0))

In [19]:
print(ptbx_from_vects)

PTBAxes(P Axis(az: 360.00°, pl: -0.00°), T Axis(az: 90.00°, pl: -0.00°))


It is possible to compare two PTBAxes instances for being almost equal: 

In [20]:
ptbx_from_vects.almostEqual(ptbx_from_axes)

True

In [21]:
ptbx_from_vects.almostEqual(ptbx_from_fltsl)

False

A PTBAxes instance can be converted to a matrix or to a quaternion, for being further processed (e.g., rotated)

In [22]:
quat = ptbx_from_vects.toQuatern()

In [23]:
print(quat)

Quaternion(1.00000, 0.00000, 0.00000, 0.00000)
